In [1]:
import requests
import pandas as pd
import time

API = "http://127.0.0.1:5000"

def get_basic_income_amounts(policy_id):
    # API endpoint URL
    api_url = f"{API}/us/policy/{policy_id}"

    # Making a GET request to the API
    response = requests.get(api_url)

    if response.status_code == 200:
        # Parsing the JSON response
        policy_data = response.json()

        # Extracting and renaming the basic income amounts
        basic_income_amounts = {}
        age_group_mapping = {
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount": "ubi_0_5",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount": "ubi_6_17",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount": "ubi_18_64",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount": "ubi_65_plus",
            "gov.contrib.ubi_center.basic_income.amount.person.disability": "ubi_disability"
        }

        for key, value in policy_data["result"]["policy_json"].items():
            if key in age_group_mapping:
                # Assuming the amount is the first value in the dictionary
                amount = next(iter(value.values()))
                basic_income_amounts[age_group_mapping[key]] = amount
        return basic_income_amounts
    else:
        return None

# Read the CSV file into a DataFrame
df = pd.read_csv('optimisation_results.csv')

def get_json_from_id(id):
    res = requests.get(f"{API}/us/economy/{id}/over/2?time_period=2024&region=enhanced_us").json()

    while res["status"] == "computing":
        time.sleep(5)
        res = requests.get(f"{API}/us/economy/{id}/over/2?time_period=2024&region=enhanced_us").json()
    
    return res["result"]

# Fetch and add basic income amounts for each reform ID
json_data = {}

from tqdm import tqdm

for i, row in tqdm(df.iterrows(), total=len(df)):
    reform_id = int(row['rounded_ubi_api_id'])
    amounts = get_basic_income_amounts(reform_id)
    if amounts:
        for column_name, amount in amounts.items():
            df.loc[i, column_name] = amount
    json_data[reform_id] = get_json_from_id(reform_id)

df.to_csv('ubi_amounts.csv', index=False)

100%|██████████| 11/11 [38:37<00:00, 210.65s/it]


In [5]:
df = pd.read_csv('ubi_amounts.csv')

import json
from policyengine_core.charts import format_fig

# Write the dictionary to disk as JSON
with open("json_data.json", "w") as file:
    json.dump(json_data, file)
import plotly.express as px

# Melt the DataFrame to make it suitable for Plotly
melted_df = df.sort_values("flat_tax_rate").melt(id_vars=['flat_tax_rate'], 
                    value_vars=['ubi_0_5', 'ubi_6_17', 'ubi_18_64', 'ubi_65_plus', 'ubi_disability'],
                    var_name='UBI Category', value_name='Amount')

# Create the plot
fig = px.line(melted_df, x='flat_tax_rate', y='Amount', color='UBI Category',
              labels={'flat_tax_rate': 'Flat Tax Rate', 'Amount': 'UBI Amount'},
              title='UBI Amounts by Flat Tax Rate')

# Show the plot
format_fig(fig)

In [6]:
import pandas as pd

def create_dataframes_from_json(json_data, df):
    dataframes = {}
    for api_id, metrics in json_data.items():
        for metric_name, metric_data in metrics.items():
            if metric_name not in dataframes:
                dataframes[metric_name] = pd.DataFrame()

            # Flatten the metric data and add API ID
            flat_data = pd.DataFrame([{**{"rounded_ubi_api_id": api_id}, **metric_data}])
            dataframes[metric_name] = pd.concat([dataframes[metric_name], flat_data], ignore_index=True)

    # Ensure data type consistency for merging
    df['rounded_ubi_api_id'] = df['rounded_ubi_api_id'].astype(str)

    # Merge each dataframe with the df dataframe to add the flat_tax_rate
    for metric_name, metric_df in dataframes.items():
        metric_df['rounded_ubi_api_id'] = metric_df['rounded_ubi_api_id'].astype(str)
        merged_df = metric_df.merge(df[['rounded_ubi_api_id', 'flat_tax_rate']], on='rounded_ubi_api_id', how='left')
        # Sort the DataFrame by flat_tax_rate
        dataframes[metric_name] = merged_df.sort_values(by='flat_tax_rate')

    return dataframes

# Assuming 'json_data' is your JSON data and 'df' is your existing DataFrame
dfs = create_dataframes_from_json(json_data, df)

# Accessing and printing a specific dataframe, for example, 'budget'
budget_df = dfs.get('budget', None)
if budget_df is not None:
    print(budget_df)


   rounded_ubi_api_id  baseline_net_income  benefit_spending_impact  \
0               43085         1.198023e+13             4.635617e+11   
1               43086         1.198023e+13             5.929647e+11   
2               43087         1.198023e+13             7.424420e+11   
3               43088         1.198023e+13             8.725101e+11   
4               43089         1.198023e+13             1.005590e+12   
5               43090         1.198023e+13             1.135097e+12   
6               43091         1.198023e+13             1.255354e+12   
7               43092         1.198023e+13             1.403570e+12   
8               43093         1.198023e+13             1.524120e+12   
9               43094         1.198023e+13             1.658476e+12   
10              43095         1.198023e+13             1.797440e+12   

    budgetary_impact    households  state_tax_revenue_impact  \
0      -3.560157e+09  1.346633e+08                       0.0   
1       3.355840e+0

In [10]:
import pandas as pd
import plotly.express as px

def calculate_percentage_change(dataframe, column_name):
    data = dataframe[column_name].apply(pd.Series)
    data['% change'] = ((data['reform'] - data['baseline']) / data['baseline'])
    data['flat_tax_rate'] = dataframe['flat_tax_rate']
    return data[['flat_tax_rate', '% change']].rename(columns={'% change': f'% change {column_name}'})

# Assuming dfs['inequality'] is your DataFrame
gini_change = calculate_percentage_change(dfs['inequality'], 'gini')
top_10_pct_share_change = calculate_percentage_change(dfs['inequality'], 'top_10_pct_share')
top_1_pct_share_change = calculate_percentage_change(dfs['inequality'], 'top_1_pct_share')

labour_supply_response = dfs["labour_supply_response"][["total_change", "flat_tax_rate"]].rename(columns=dict(total_change="labor_supply"))
labour_supply_response.labor_supply /= 10320e9
# Combine the data
combined_data = gini_change
combined_data = combined_data.merge(top_10_pct_share_change, on='flat_tax_rate')
combined_data = combined_data.merge(top_1_pct_share_change, on='flat_tax_rate')
combined_data = combined_data.merge(labour_supply_response, on='flat_tax_rate')

fig = px.line(combined_data, x='flat_tax_rate', y=['% change gini', '% change top_10_pct_share', '% change top_1_pct_share'],
              labels={'value': '% Change', 'variable': 'Metric'},
              title='Percentage Change in Inequality Metrics with Respect to Tax Rate')
format_fig(fig)

In [11]:
def calculate_percentage_change_poverty(dataframe, column_name):
    data = dataframe[column_name].apply(pd.Series)
    for category in ['adult', 'all', 'child', 'senior']:
        data[f'% change {category}'] = ((data[category].apply(lambda x: x['reform']) - 
                                         data[category].apply(lambda x: x['baseline'])) / 
                                         data[category].apply(lambda x: x['baseline']))
    data['flat_tax_rate'] = dataframe['flat_tax_rate']
    return data[['flat_tax_rate', f'% change adult', f'% change all', f'% change child', f'% change senior']]

# Assuming dfs['poverty'] is your DataFrame
poverty_change = calculate_percentage_change_poverty(dfs['poverty'], 'poverty')


fig = px.line(poverty_change, x='flat_tax_rate', y=[f'% change {category}' for category in ['adult', 'all', 'child', 'senior']],
              labels={'value': '% Change', 'variable': 'Category'},
              title='Percentage Change in Poverty Metrics with Respect to Tax Rate')
format_fig(fig)


In [12]:
deep_poverty_change = calculate_percentage_change_poverty(dfs['poverty'], 'deep_poverty')


fig = px.line(deep_poverty_change, x='flat_tax_rate', y=[f'% change {category}' for category in ['adult', 'all', 'child', 'senior']],
              labels={'value': '% Change', 'variable': 'Category'},
              title='Percentage Change in Deep Poverty Metrics with Respect to Tax Rate')
fig.show()


In [13]:
labor_supply_change = combined_data[["flat_tax_rate", "labor_supply"]]

px.line(
    combined_data,
    x="flat_tax_rate",
    y="labor_supply"
)

In [14]:
import pandas as pd
import plotly.express as px

# Define a function to determine increase (1) or decrease (-1)
def increase_or_decrease(value):
    return 1 if value > 0 else -1
        

# Combine the percentage change data from poverty, deep poverty, and inequality
# Assuming dfs['poverty'] and other DataFrames are already defined
combined_change = dfs['poverty'][['flat_tax_rate']].copy()
combined_change_categorical = combined_change.copy()

# Assume you have other dataframes like gini_change, top_10_pct_share_change, etc.
for dataframe in [labor_supply_change, gini_change, top_10_pct_share_change, top_1_pct_share_change, poverty_change, deep_poverty_change]:
    dataframe.sort_values('flat_tax_rate', inplace=True)
    for col in dataframe.columns:
        if col != 'flat_tax_rate':
            if col == "labor_supply":
                combined_change_categorical[col] = dataframe[col].apply(lambda x: -1 if x > 0 else 1)
            else:
                combined_change_categorical[col] = dataframe[col].apply(increase_or_decrease)
            combined_change[col] = dataframe[col]

import pandas as pd
import plotly.figure_factory as ff

# Assume combined_change is a DataFrame with the given structure
# Mapping the percentage change to colors
combined_change_mapped = combined_change_categorical.copy()

import pandas as pd
import plotly.figure_factory as ff
import numpy as np

# Rest of your code where you define combined_change and combined_change_mapped...

# Preparing the data for the heatmap
heatmap_data = combined_change_mapped.set_index('flat_tax_rate').T

# The x and y coordinates for the heatmap should be lists or arrays
x_values = combined_change_mapped['flat_tax_rate'].tolist()
y_values = heatmap_data.index.tolist()

# The z values and annotations must be 2D arrays that match the length of x and y
#heatmap_data = heatmap_data.replace({'Increase': "+", 'Decrease': "-"})
z_values = heatmap_data.values.T  # Transposing to match the dimensions
annotations = combined_change.set_index('flat_tax_rate').T.values.T  # This should be the same shape as z_values

heatmap_annotations = pd.DataFrame(annotations, columns=y_values, index=x_values)
heatmap_annotations = heatmap_annotations.apply(lambda x: x.apply(lambda y: f"{y:+.0%}"))
# Assume combined_change is a DataFrame with the given structure
# Mapping the percentage change to colors

from policyengine_core.charts import *
y_values = [
    "Labor supply",
    "Gini index",
    "Top 10% income share",
    "Top 1% income share",
    "Adult poverty rate",
    "Poverty rate",
    "Child poverty rate",
    "Senior poverty rate",
]

# Creating the heatmap using Figure Factory to properly display categorical data
fig = ff.create_annotated_heatmap(
    z=z_values.T,  # Transposing to match the dimensions
    x=x_values,
    y=y_values,
    annotation_text=heatmap_annotations.values.T,
    colorscale=[BLUE, DARK_GRAY],
    showscale=False
)

# Update the layout to remove the color axis scale (legend)
format_fig(fig).update_layout(
    title='Change in metric under optimal UBI by flat tax rate',
    xaxis=dict(title='Flat Tax Rate', side='top', tickmode='array', tickvals=x_values, ticktext=[f"{x:.0%}" for x in x_values]),
    yaxis=dict(title='Metric', tickmode='array', tickvals=list(range(len(y_values))), ticktext=y_values),
    margin=dict(l=150, t=150),  # Adjust margins to fit y-axis labels
    height=600,
    width=1200
)

# Show the figure
fig.show()


/var/folders/8f/pgfhysmd5ls3jnxb_5j7yy340000gn/T/ipykernel_72236/1721656161.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

